# Final Submission to Phyla Challenge Two : Multi-Disease Classification

## Reading Data

In [ ]:
import pandas as pd

data = pd.read_csv('/content/sample_data/challenge_1_gut_microbiome_data.csv', error_bad_lines = False)
data.head()

## Pre-Processing Data

SMOTE

In [ ]:
from imblearn.over_sampling import SMOTE

smote = SMOTE()
bacteria_names= list(data.columns)[1:-1]
all_bacteria, all_diseases = smote.fit_resample(data[bacteria_names], data['disease'])

One-Hot Encoded Y

In [ ]:
from sklearn.preprocessing import LabelEncoder
from keras.utils import np_utils

encoder=LabelEncoder()
encoder.fit(all_diseases)
encoded_Y=encoder.transform(all_diseases)
dummy_y = np_utils.to_categorical(encoded_Y)

Standard Scaler

In [ ]:
from sklearn import preprocessing

scaler = preprocessing.StandardScaler()
all_bacteria_scaled= scaler.fit_transform(all_bacteria)
all_bacteria_scaled

## Sequential Neural Network

In [ ]:
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier


model = Sequential()
model.add(Dense(500, input_dim=1094, activation='relu'))
model.add(Dense(175, activation='relu'))
model.add(Dense(75, activation='relu'))
model.add(Dense(4, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Final results

Convert List of probabilities to the Desired Disease

In [ ]:
def simplify_softmax (results) :
  
  new=[]
  diseases=['Disease-1','Disease-2','Disease-3','Healthy']
  for result in results:
    l=list(result)
    ind = l.index(max(l))
    new.append(diseases[ind])
  return new

Cross Validation of Model

In [ ]:
from sklearn.metrics import cohen_kappa_score
cv_scores=[]
ck=[]
for i in range(10):
  model = Sequential()
  model.add(Dense(500, input_dim=1094, activation='relu'))
  model.add(Dense(175, activation='relu'))
  model.add(Dense(75, activation='relu'))
  model.add(Dense(4, activation='softmax'))
  model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

  X_train, X_test, y_train, y_test = train_test_split(all_bacteria_scaled, dummy_y, test_size=0.25)
  model.fit(X_train, y_train, epochs=50)
  pred_test=model.predict(X_test)
  scores2 = model.evaluate(X_test, y_test, verbose=0)
  cv_scores.append(scores2[1]*100)

  y_pred = simplify_softmax(pred_test)
  y_test1 = simplify_softmax(y_test) 

  ck.append(cohen_kappa_score(y_test1,y_pred))


F1_Score

In [ ]:
import matplotlib.pyplot as plt

plt.boxplot(cv_scores)
plt.show()

Cohen_Kappa Score

In [ ]:

plt.boxplot(ck)
plt.show()